Code used to create result file for tying out decision variable, objective function and constraint code.

In [14]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_results

In [15]:
#get the distinct list of city and demand type combinations

list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
print(list_city_dt)
print(len(list_city_dt))

['lyon_db=0.50', 'paris_db=1.00', 'lyon_db=4.00', 'berlin_db=2.00', 'paris_db=2.00', 'frankfurt_db=4.00', 'frankfurt_db=2.00', 'berlin_db=4.00', 'berlin_db=1.00', 'lyon_db=2.00', 'frankfurt_db=0.50', 'lyon_db=1.00', 'berlin_db=0.50', 'frankfurt_db=1.00', 'paris_db=4.00', 'paris_db=0.50']
16


In [16]:
#create a dataframe that can hold all result information

list_results = []

workforce_dict = {0:5, 1:5, 2:5, 3:5, 4:5}
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

for base_file in list_city_dt:
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"
    
    # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for OC in [1.5]:
        model = 'fixed'
        dict_results = run_roster_solver_results(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
        df_ = pd.DataFrame(dict_results)
        list_results.append(df_)
        model = 'partflex'
        max_n_shifts = 3
        dict_results = run_roster_solver_results(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
        df_ = pd.DataFrame(dict_results)
        list_results.append(df_)


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28760 rows, 31680 columns and 141396 nonzeros
Model fingerprint: 0x03ebe34a
Variable types: 26880 continuous, 4800 integer (4800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 28085 rows and 29529 columns
Presolve time: 0.14s
Presolved: 675 rows, 2151 columns, 5441 nonzeros
Variable types: 656 continuous, 1495 integer (1495 binary)
Found heuristic solution: objective 3196.5041667
Found heuristic solution: objective 3192.6041667

Root relaxation: objective 2.845729e+03, 1929 iteratio

In [17]:
df_results = pd.concat(list_results, ignore_index = True)
df_results.drop(columns = ['wage_costs','objective_value_post_wage'], inplace = True)
df_results.sort_values(by = ['city','demand_baseline','model'], inplace = True)

df_results.to_csv(r'../rostering_results/results_cm.csv', index = False)

In [11]:
for file_in in os.listdir(r'../results'):
    if 'jonny' in file:
        print(file_in)